In [1]:
import numpy as np 
import pandas as pd
import requests
from groq import Groq
from datetime import datetime
import calendar
import matplotlib.pyplot as plt
import math
from openai import OpenAI
import time

In [42]:
contracts = pd.read_csv('/home/xikron/Projects/misc/data/contracts_data.csv')
contracts_test = contracts[contracts["Start Date"].str.match(r"^\d{4}-\d{2}-\d{2}$", na=False)]
contracts_test["Start Date"] = pd.to_datetime(contracts_test["Start Date"])
contracts_test = contracts_test[
    (contracts_test["Start Date"] >= pd.Timestamp("2016-01-01")) & 
    (contracts_test["Start Date"] < pd.Timestamp("2019-09-01")) ]
contracts_test = contracts_test.sort_values(by="Start Date")

/tmp/ipykernel_243926/3517862119.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contracts_test["Start Date"] = pd.to_datetime(contracts_test["Start Date"])


In [43]:
capital = 10000000
portfolio = {}

In [44]:
def setLink (ticker, start, end, freq):
    link="https://financialmodelingprep.com/api/v3/historical-chart/"+freq+"/"+ticker+"?from="+start+"&to="+end+"&apikey=26srycwxWrFIhEuaZwic6mBdx7f4VjGT"
    return link

In [45]:
from dataclasses import dataclass

@dataclass
class transaction:
    contractID: int
    num: int

In [46]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

def getBuyDay(contractDate):
    uBday = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    contractDate = pd.Timestamp(contractDate)
    buyDay = contractDate - 3 * uBday
    return buyDay.strftime('%Y-%m-%d')

def getSellDay(contractDate):
    uBday = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    contractDate = pd.Timestamp(contractDate)
    sellDay = contractDate + 5*uBday
    return sellDay.strftime('%Y-%m-%d')

In [47]:
def buy(ticker, valuation, buyDate, id): 
    global portfolio, capital
    response = requests.get(url=setLink(ticker, buyDate, buyDate, "5min"))
    if response.status_code == 200: 
        pps = response.json()[-15]["open"]
        shares = valuation/pps
        if ticker not in portfolio: 
            portfolio[ticker]= [transaction(contractID=id, num=shares)] 
        else: 
            portfolio[ticker].append(transaction(contractID=id, num=shares))
        capital-=shares*pps 
        

In [48]:
def sell(ticker, sellDate, id):
    global portfolio, capital
    response = requests.get(url=setLink(ticker, sellDate, sellDate, "5min"))
    if response.status_code == 200: 
        pps = response.json()[-15]["open"]
        for t in portfolio[ticker]: 
            if t.contractID == id: 
                shares = t.num
                value = shares*pps
                capital+=value 
                portfolio.remove(t)
                break

In [ ]:
for 